In [1]:
import pandas as pd

# URL en modo export CSV
url = "https://docs.google.com/spreadsheets/d/1YtUaTmcVQR7N9FvXb5mYmvrUKkOnufwQV9HoF9Gf75I/export?format=csv&gid=0"

# Leer datos
df = pd.read_csv(url)

print("Filas cargadas:", len(df))
print(df.head())


Filas cargadas: 154
            nombre           proyecto producto                parte  \
0      ELKIN PARRA    25-0103 ARCADIA   PUERTA  LISTONES ESTRUCTURA   
1     JAIRO FUENTE       00-0000 WLZF   PUERTA             TABLEROS   
2     FUAD AGUILAR    25-0103 ARCADIA   PUERTA             TABLEROS   
3  ORLANDO CABRERA    25-0103 ARCADIA   PUERTA             TABLEROS   
4      ELKIN PARRA  25-0306 THE WOODS   PUERTA  LISTONES ESTRUCTURA   

        ruta    maquina    proceso  piezas         fecha_inicio  \
0    PINTURA     CABINA     LACADO       2   22/6/2025, 9:39:30   
1    PINTURA     CABINA     LACADO       5   22/6/2025, 9:45:26   
2  MAQUINADO  CNC ROVER  PERFORADO       7   22/6/2025, 9:51:29   
3  MAQUINADO    CEPILLO     GRUESO       2  2025-06-22 12:53:14   
4  MAQUINADO    CEPILLO     GRUESO      35  2025-06-22 13:01:01   

             fecha_fin  
0   22/6/2025, 9:39:32  
1   22/6/2025, 9:48:37  
2   22/6/2025, 9:51:38  
3  2025-06-22 12:53:18  
4  2025-06-22 13:02:51  


In [ ]:
import numpy as np

# Definir el rango de hora de almuerzo (por ejemplo, de 12:00 a 13:00)
almuerzo_inicio = pd.to_datetime("12:00:00").time()
almuerzo_fin = pd.to_datetime("13:00:00").time()

# Convertir las columnas de fecha a datetime
df['fecha_inicio_dt'] = pd.to_datetime(df['fecha_inicio'], errors='coerce')
df['fecha_fin_dt'] = pd.to_datetime(df['fecha_fin'], errors='coerce')

def calcular_almuerzo(row):
    # Si alguna fecha es NaT, retornar 0
    if pd.isnull(row['fecha_inicio_dt']) or pd.isnull(row['fecha_fin_dt']):
        return 0
    # Crear rango de almuerzo en el mismo día que fecha_inicio
    almuerzo_start = row['fecha_inicio_dt'].replace(hour=almuerzo_inicio.hour, minute=almuerzo_inicio.minute, second=almuerzo_inicio.second)
    almuerzo_end = row['fecha_inicio_dt'].replace(hour=almuerzo_fin.hour, minute=almuerzo_fin.minute, second=almuerzo_fin.second)
    # Calcular intersección
    inicio = max(row['fecha_inicio_dt'], almuerzo_start)
    fin = min(row['fecha_fin_dt'], almuerzo_end)
    delta = (fin - inicio).total_seconds() / 60  # minutos
    return max(0, delta)

df['min_almuerzo'] = df.apply(calcular_almuerzo, axis=1)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23648\3312547084.py:8: UserWarning: Parsing dates in %d/%m/%Y, %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['fecha_inicio_dt'] = pd.to_datetime(df['fecha_inicio'], errors='coerce')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_23648\3312547084.py:9: UserWarning: Parsing dates in %d/%m/%Y, %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['fecha_fin_dt'] = pd.to_datetime(df['fecha_fin'], errors='coerce')


In [5]:
# Calcular la diferencia en minutos entre fecha_inicio_dt y fecha_fin_dt, restando min_almuerzo
def calcular_minutos(row):
    if pd.isnull(row['fecha_inicio_dt']) or pd.isnull(row['fecha_fin_dt']):
        return 0
    total_min = (row['fecha_fin_dt'] - row['fecha_inicio_dt']).total_seconds() / 60
    return max(0, total_min - row['min_almuerzo'])

df['minutos'] = df.apply(calcular_minutos, axis=1)

In [12]:
df.head(2)

,nombre,proyecto,producto,parte,ruta,maquina,proceso,piezas,fecha_inicio,fecha_fin,fecha_inicio_dt,fecha_fin_dt,min_almuerzo,minutos,fuera_oficina
0,ELKIN PARRA,25-0103 ARCADIA,PUERTA,LISTONES ESTRUCTURA,PINTURA,CABINA,LACADO,2,"22/6/2025, 9:39:30","22/6/2025, 9:39:32",2025-06-22 09:39:30,2025-06-22 09:39:32,0,0.033333,0
1,JAIRO FUENTE,00-0000 WLZF,PUERTA,TABLEROS,PINTURA,CABINA,LACADO,5,"22/6/2025, 9:45:26","22/6/2025, 9:48:37",2025-06-22 09:45:26,2025-06-22 09:48:37,0,3.183333,0


In [ ]:
# Definir hora de salida de oficina (17:30)
hora_salida = pd.to_datetime("17:30:00").time()

def calcular_fuera_oficina(row):
    if pd.isnull(row['fecha_fin_dt']):
        return 0
    fin = row['fecha_fin_dt'].time()
    # Si la hora de fin es después de 17:30, marcar como 1 (hay tiempo fuera de oficina)
    return int(fin > hora_salida)

df['fuera_oficina'] = df.apply(calcular_fuera_oficina, axis=1)

In [14]:
def ajustar_minutos_fuera_oficina(row):
    if row['fuera_oficina'] == 1 and not pd.isnull(row['fecha_inicio_dt']) and not pd.isnull(row['fecha_fin_dt']):
        # Limitar fecha_fin_dt a las 17:30 del mismo día
        fecha_salida = row['fecha_fin_dt'].replace(hour=hora_salida.hour, minute=hora_salida.minute, second=0)
        total_min = (fecha_salida - row['fecha_inicio_dt']).total_seconds() / 60
        # Restar min_almuerzo si corresponde
        return max(0, total_min - row['min_almuerzo'])
    else:
        return row['minutos']

df['minutos_ajustados'] = df.apply(ajustar_minutos_fuera_oficina, axis=1)

In [8]:
df.columns

Index(['nombre', 'proyecto', 'producto', 'parte', 'ruta', 'maquina', 'proceso',
       'piezas', 'fecha_inicio', 'fecha_fin', 'fecha_inicio_dt',
       'fecha_fin_dt', 'min_almuerzo', 'minutos'],
      dtype='object')